In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pie Chart Import
import plotly.express as px

# URL Lib to make sure that our input is 'sane'
import urllib.parse


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import driver
from pymongo import MongoClient

#callback_context for recieving and validating user credentials
from dash import callback_context

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "accuser"
password = "Zelda5Persona5RoyalofTime"
host = "nv-desktop-services.apporto.com"
port = 31714
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
#db = driver.AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame(columns=['animal_type', 'breed', 'sex_type', 'age_upon_outcome_in_weeks',
                          'name', 'date_of_birth', 'outcome_type', 'color', 'location_lat', 'location_long'
                          ])

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'assets/Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center([
    html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='100px'),
        href="https://www.snhu.edu",
        target='blank' #Opens the link in a blank new tab
    ),
    html.B(html.H1('CS-340 Dashboard'))]),
    # This Input statement sets up an Input field for the username.
    dcc.Input(
            id="input_user",
            type="text",
            placeholder="Username"),
    # This Input statement sets up an Input field for the password.
    # This designation masks the user input on the screen.
    dcc.Input(
            id="input_passwd",
            type="password",
            placeholder="Password"),
    # Create a button labeled 'Submit'. When the button is pressed
    # the n_clicks value will increment by 1. 
    html.Button('Submit', id='submit-val', n_clicks=0),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        html.Label("Rescue Type:"),
        dcc.RadioItems(
            id='rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'},
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
            
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         editable=False,
                         filter_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
        html.Hr(style={"border": "none", "borderTop": "3px solid #bbb", "margin": "20px 0"})
        ]),
        html.Div(
            id='author',
            className='col s12 m6'
        ),
        html.H1("Ishmael K. Kwayisi"),
        html.Div(
            id='unique_id',
            className='col s12 m6'
        ),
        html.Img(src="/assets/This_Is_My_Lab.jpg", style={"width" : "200px"})
])

#############################################
# Interaction Between Components / Controller
#############################################
# Define callback to update output-block
# NOTE: While the name of the callback function doesn't matter,
# the order of the parameters in the callback function are the
# same as the order of Input methods in the @app.callback
# For the callback function below, the callback is grabing the
# information from the input_user and input_password entries, and
# then the value of the submit button (has it been pressed?)

@app.callback(Output('datatable-id','data'),
                [Input('rescue-type', 'value'),
                Input('input_user', 'value'),
                Input('input_passwd', 'value'),
                Input(component_id='submit-val', component_property='n_clicks')])
              
def update_dashboard(rescue_type, inputUser,inputPass,n_clicks):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        

    #n_clicks needs to be at least 1 to display information once user 
    #credentials have been submitted
    if n_clicks > 0:
        ###########################
        # Data Manipulation / Model
        # use CRUD module to access MongoDB
        ##########################
        
        # Use the URLLIB to setup the username and password so that they
        # can be passed cleanly to the MongoDB handler.
        #username = urllib.parse.quote_plus(inputUser)
        #password = urllib.parse.quote_plus(inputPass)
        
        username = inputUser
        password = inputPass
        
        ## DEBUG STATEMENT - You can uncomment the next line to verify you
        ## are correctly entering your username and password prior to continuing
        ## to build the callback function.
        ##return f'Output: {inputUser}, {inputPass}'
        try:
            print(f"Trying to authenticate with: {username}, {password}")
            # Instantiation of CRUD object
            shelter = driver.AnimalShelter(username, password, host, port, database, collection)
            df = pd.DataFrame.from_records(shelter.read({}))
              
            #Select a radio button once to recieve filter information
            if rescue_type == "water":
                df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                                                            "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":
                                                            {"$gte":26, "$lte":156}}))
            elif rescue_type == "mountain":
                df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                                                            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":
                                                            {"$gte":26,"$lte":156}}))
            elif rescue_type == "disaster":
                df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever", "Bloodhound", "Rottweiler"]},
                                                            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":
                                                            {"$gte":20, "$lte":300}}))
            elif rescue_type == "reset":
                df = pd.DataFrame.from_records(shelter.read({}))

            # Display the breeds of animal based on quantity represented in
            # the data table
            # Cleanup Mongo _id field
            if '_id' in df.columns:
                df.drop(columns=['_id'], inplace=True)

            return df.to_dict('records')

            
        except Exception as e:
            print(f"Authentication failed: {e}")
            return []


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return html.P("No data to display!")
    
    #This is to convert the incoming data (dict) into a DataFrame
    df = pd.DataFrame(viewData)
    
    #Checks to see if there is any data in 'breed' column before displaying information in the 
    #pie chart
    if "breed" not in df.columns:
        return [html.Div([html.P("No column labeled 'breed' exists in the data!"), 
                html.Hr(style={"border": "none", "borderTop": "3px solid #bbb", "margin": "20px 0"})])]
    
    # Count the number of animals per breed
    breed_count = df["breed"].value_counts().reset_index()
    breed_count.columns = ["breed", "count"]
    
    #fig = px.pie(
    #)
    
    return [
        dcc.Graph(            
            figure = px.pie(breed_count, names='breed', values='count', title='Preferred Animals')
        )    
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:26874/
